## Thông tin nhóm

**Thành viên thứ 1:**

- Họ và tên: Trần Trung Hiếu
- MSSV: 1712442

**Thành viên thứ 2:**
- Họ và tên: Lê Long Hồ
- MSSV: 1712447

**Thành viên thứ 3:**
- Họ và tên: Nguyễn Đình Thiên Phúc
- MSSV: 18120144

___

## Import

In [122]:
!pip3 install nltk
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import csv
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

from sklearn.model_selection import train_test_split
import re
pd.set_option('display.max_rows', 500)

  Using cached https://files.pythonhosted.org/packages/5a/75/aca08032c9752a75acc60ff7f4e58f1a74164b996395f44727ffdb17ebaf/regex-2020.11.13-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/34/5b/bd0f0fb5564183884d8e35b81d06d7ec06a20d1a0c8b4c407f1554691dce/joblib-1.0.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d2/3d/fa76db83bf75c4f8d338c2fd15c8d33fdd7ad23a9b5e57eb6c5de26b430e/click-7.1.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/80/02/8f8880a4fd6625461833abcf679d4c12a44c76f9925f92bf212bb6cefaad/tqdm-4.56.0-py2.py3-none-any.whl


[nltk_data] Downloading package punkt to
[nltk_data]     /home/tthieu22_06/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


---

## Thu thập dữ liệu

TODO

---

## Khám phá dữ liệu

In [23]:
movie_df = pd.read_csv('movie_final.csv')
movie_df.sample(5)

,url,title,score,posterImagePath,certification,overview,tagline,genres,trailerUrl,language
1660,https://www.themoviedb.org/tv/113879,Beyond the Clouds\n(2020),0.0,NaN,NaN,Beyond the Clouds takes an exclusive look insi...,NaN,Phim Tài Liệu,NaN,en
1642,https://www.themoviedb.org/tv/107853,Reino Animal ou Bichikids\n(2020),0.0,NaN,NaN,We don't have an overview translated in Englis...,NaN,NaN,NaN,en
2381,https://www.themoviedb.org/tv/34793,Girls Bravo\n(2004),77.0,poster_final/34793.jpg,NaN,"A high school boy, Yukinari, has been bullied ...",NaN,"Phim Hoạt Hình, Sci-Fi & Fantasy, Phim Hài",NaN,en
3883,https://www.themoviedb.org/movie/596247,Pacto de Fuga\n(2020),83.0,poster_final/596247.jpg,PG,During the last years of Pinochet's military r...,NaN,"Phim Chính Kịch, Phim Gây Cấn",NaN,en
2890,https://www.themoviedb.org/movie/9651,Supergirl\n(1984),54.0,poster_final/9651.jpg,PG,"After losing a powerful orb, Kara, Superman's ...",From out of another galaxy and into your heart...,"Phim Phiêu Lưu, Phim Giả Tượng, Phim Hành Động...",NaN,en


In [24]:
movie_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5598 entries, 0 to 5597
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   url              5598 non-null   object 
 1   title            5594 non-null   object 
 2   score            5594 non-null   float64
 3   posterImagePath  5326 non-null   object 
 4   certification    4177 non-null   object 
 5   overview         5594 non-null   object 
 6   tagline          2522 non-null   object 
 7   genres           5230 non-null   object 
 8   trailerUrl       79 non-null     object 
 9   language         5594 non-null   object 
dtypes: float64(1), object(9)
memory usage: 437.5+ KB


In [25]:
n_row, n_col = movie_df.shape
print("Dữ liệu có số dòng:", n_row)
print("Dữ liệu có số cột:", n_col)

Dữ liệu có số dòng: 5598
Dữ liệu có số cột: 10


Mỗi dòng trong tập dữ liệu thể hiện vài thông tin của một bộ phim, bộ phim đó có thể là phim truyền hình hoặc phim điện ảnh. Các dòng này đều có ý nghĩa tương đương nhau. 

## Đưa ra câu hỏi cần trả lời

Các thông tin được tải lên trên Internet hầu như không được kiểm duyệt, chính vì vậy sẽ có thể xuất hiện những nội dung không phù hợp với mọi lứa tuổi, cụ thể là những bộ phim có gán nhãn giới hạn độ tuổi người lớn không nên để những người chưa đủ tuổi, đặc biệt là trẻ nhỏ tiếp xúc. Tuy nhiên, rất nhiều các website xem phim lại không có gán nhãn giới hạn độ tuổi. Chính vì vậy, câu hỏi này được đưa ra nhằm tìm câu trả lời giải quyết vấn đề nội dung bộ phim không phù hợp với các lứa tuổi, giúp các phụ huynh có thể quản lí được trẻ nhỏ không tìm coi những bộ phim có nội dung không phù hợp.

Câu hỏi này sẽ liên quan tới việc phân loại nội dung tóm tắt của một bộ phim và dự đoán độ tuổi phù hợp để coi bộ phim này là gì.

- Input: overview (nội dung tóm tắt của bộ phim)
- Output: certification (độ tuổi phù hợp)

---

## Khám phá dữ liệu

In [26]:
print("Danh sách các ngôn ngữ thể hiện nội dung tóm tắt:")
print(movie_df.language.value_counts())

Danh sách các ngôn ngữ thể hiện nội dung tóm tắt:
en    4304
vi    1224
lt      31
fi      10
tl       5
pt       4
cy       4
ro       3
es       2
de       1
fr       1
it       1
af       1
cs       1
id       1
sk       1
Name: language, dtype: int64


In [27]:
print('Kiểu dữ liệu của cột cerfitication:', movie_df.certification.dtype)

Kiểu dữ liệu của cột cerfitication: object


In [28]:
print("Số lượng dòng bị thiếu output:", movie_df.certification.isnull().sum())
print("Số lượng dòng có output:", movie_df.shape[0] - movie_df.certification.isnull().sum())

Số lượng dòng bị thiếu output: 1421
Số lượng dòng có output: 4177


In [29]:
print('Số lượng lớp trong certification:', len(movie_df.certification.unique()))
print('Tỉ lệ bộ phim của từng lớp:\n', movie_df.certification.value_counts(normalize=True)*100)

Số lượng lớp trong certification: 88
Tỉ lệ bộ phim của từng lớp:
 R             14.891070
TV-14         12.401245
PG-13         10.892985
PG             9.743835
TV-MA          9.743835
TV-PG          4.907829
G              4.141729
12             4.045966
NR             3.998085
15             3.303807
16             2.657410
TV-Y7          2.250419
TV-G           1.723725
U              1.484319
18             1.173091
14             1.125209
L              1.053388
16+            0.694278
10             0.598516
0              0.598516
6              0.550634
18+            0.478813
T              0.430931
12+            0.406991
MA15+          0.383050
保護級            0.359109
TV-Y           0.359109
普遍級            0.335169
AL             0.335169
6+             0.287288
7              0.287288
輔12級           0.239406
UA             0.215466
M              0.215466
NC-17          0.215466
APTA           0.215466
11             0.191525
M/12           0.167584
15세 이상 관람가     0.14364

## Tiền xử lí - tách các tập dữ liệu

### Lọc thông tin cần thiết

`en` và `vi` là hai ngôn ngữ có số lượng bộ phim chiếm nhiều nhất, cho nên nhóm sẽ giữ lại những bộ phim thuộc một trong hai ngôn ngữ này để thực hiện tiền xử lí và loại bỏ đi các dòng chứa các ngôn ngữ còn lại

In [30]:
envi_movie_df = movie_df[((movie_df.language == 'vi') | (movie_df.language == 'en'))]
envi_movie_df.reset_index(inplace=True, drop=True)
envi_movie_df.head(5)

,url,title,score,posterImagePath,certification,overview,tagline,genres,trailerUrl,language
0,https://www.themoviedb.org/tv/44217,Huyền Thoại Vikings\n(2013),79.0,poster_final/44217.jpg,14+,Người Viking đưa chúng ta đến thế giới tàn khố...,NaN,"Action & Adventure, Phim Chính Kịch",NaN,vi
1,https://www.themoviedb.org/tv/82856,The Mandalorian\n(2019),85.0,poster_final/82856.jpg,TV-14,"After the fall of the Galactic Empire, lawless...",Bounty hunting is a complicated profession.,"Sci-Fi & Fantasy, Action & Adventure",NaN,en
2,https://www.themoviedb.org/tv/77169,Cobra Kai\n(2018),81.0,poster_final/77169.jpg,TV-14,This Karate Kid sequel series picks up 30 year...,Cobra Kai never dies.,"Action & Adventure, Phim Chính Kịch",NaN,en
3,https://www.themoviedb.org/tv/75006,Học Viện Umbrella\n(2019),87.0,poster_final/75006.jpg,TV-MA,"Được đưa về quá khứ những năm 1960 tại Dallas,...",Super. Dysfunctional. Family.,"Action & Adventure, Sci-Fi & Fantasy, Phim Chí...",NaN,vi
4,https://www.themoviedb.org/tv/71712,Bác Sĩ Thiên Tài\n(2017),86.0,poster_final/71712.jpg,TV-14,Một bác sĩ trẻ mắc hội chứng bác học được nhận...,"His mind is a mystery, his methods are a miracle.",Phim Chính Kịch,NaN,vi


Xóa các dòng bị thiếu nhãn `certification`

In [31]:
notnull_envi_movie_df = envi_movie_df.dropna(subset=['certification']).reset_index(drop=True)
print('Số lượng bộ phim sau khi loại bỏ nhãn thiếu:', notnull_envi_movie_df.shape[0])

Số lượng bộ phim sau khi loại bỏ nhãn thiếu: 4171


Vì số lượng lớp lớn và phân bố không đồng đều, thực hiện chia các lớp giới hạn tuổi thành 2 cụm là: lớn hơn 18 (>=18) và nhỏ hơn 18 (<18)

Danh sách cụm này được lưu trong tệp `age_restricted.txt`

In [32]:
# Danh sách các nhóm tuổi
age_cert = {}
with open('age_restricted.txt', 'r') as cert_file:
    for line in cert_file:
        cert, age = line.strip().split(',')
        age_cert[cert] = 1 if age == '>18' else 0

notnull_envi_movie_df['is_adult'] = notnull_envi_movie_df.certification.apply(lambda c: age_cert[c])

In [33]:
notnull_envi_movie_df.sample(5)

,url,title,score,posterImagePath,certification,overview,tagline,genres,trailerUrl,language,is_adult
3282,https://www.themoviedb.org/movie/335,Miền Viễn Tây Thuở Ấy\n(1968),83.0,poster_final/335.jpg,PG-13,Một thị trấn nhỏ ở miền Tây bắt đầu trở nên bấ...,There were three men in her life. One to take ...,Phim Miền Tây,NaN,vi,0
3958,https://www.themoviedb.org/movie/26280,I Killed My Mother\n(2009),76.0,poster_final/26280.jpg,14A,"Hubert, a brash 17-year-old, is confused and t...",Sons forget their mothers can die,Phim Chính Kịch,NaN,en,0
4005,https://www.themoviedb.org/movie/57158,Người Hobbit 2: Đại Chiến Với Rồng Lửa\n(2013),76.0,poster_final/57158.jpg,PG-13,"Phim ""Người Hobbit: Đại Chiến Với Rồng Lửa"" ti...",Beyond darkness... beyond desolation... lies t...,"Phim Phiêu Lưu, Phim Giả Tượng, Phim Hành Động",NaN,vi,0
3139,https://www.themoviedb.org/movie/500854,Hard Paint\n(2018),69.0,poster_final/500854.jpg,NR,Pedro earns a living in chat rooms. He transfo...,NaN,Phim Chính Kịch,NaN,en,0
445,https://www.themoviedb.org/tv/1338,Alarm for Cobra 11: The Motorway Police\n(1996),59.0,poster_final/1338.jpg,12,A long-running German television series about ...,NaN,"Phim Chính Kịch, Phim Hài, Action & Adventure",NaN,en,0


### Tách các tập dữ liệu

Vì hai ngôn ngữ `en` và `vi` có đặc điểm khác nhau, để tránh chiều dữ liệu quá lớn và làm giảm hiệu quả của mô hình phân loại, ta sẽ chia `en` và `vi` ra làm 2 tập dữ liệu riêng biệt. Đồng thời, do chỉ tập trung vào `overview` cho nên sẽ loại bỏ các cột khác chỉ giữ lại `overview` và `is_adult`

In [83]:
en_movie_df = notnull_envi_movie_df[notnull_envi_movie_df.language == 'en'][['overview', 'is_adult']]
vi_movie_df = notnull_envi_movie_df[notnull_envi_movie_df.language == 'vi'][['overview', 'is_adult']]

len(en_movie_df), len(vi_movie_df)

(2968, 1203)

Thực hiện chia thành các tập train, valid, và test với tỉ lệ lần lượt là 70/15/15

In [108]:
# Tách tập cho tiếng Việt
vi_y_sr = vi_movie_df.is_adult
vi_X_df = vi_movie_df.drop(columns=['is_adult'])

vi_train_X, vi_tmp_X, vi_train_y, vi_tmp_y = train_test_split(vi_X_df, vi_y_sr, stratify=vi_y_sr, random_state=42, test_size=0.3)
vi_valid_X, vi_test_X, vi_valid_y, vi_test_y = train_test_split(vi_tmp_X, vi_tmp_y, stratify=vi_tmp_y, random_state=42, test_size=0.15)


In [109]:
vi_train_X.shape, vi_train_y.shape

((842, 1), (842,))

In [110]:
vi_valid_X.shape, vi_valid_y.shape

((306, 1), (306,))

In [111]:
vi_test_X.shape, vi_test_y.shape

((55, 1), (55,))

In [112]:
# Tách tập cho tiếng Anh
en_y_sr = en_movie_df.is_adult
en_X_df = en_movie_df.drop(columns=['is_adult'])

en_train_X, en_tmp_X, en_train_y, en_tmp_y = train_test_split(en_X_df, en_y_sr, stratify=en_y_sr, random_state=42, test_size=0.3)
en_valid_X, en_test_X, en_valid_y, en_test_y = train_test_split(en_tmp_X, en_tmp_y, stratify=en_tmp_y, random_state=42, test_size=0.5)

In [113]:
en_train_X.shape, en_train_y.shape

((2077, 1), (2077,))

In [114]:
en_valid_X.shape, en_valid_y.shape

((445, 1), (445,))

In [115]:
en_test_X.shape, en_test_y.shape

((446, 1), (446,))

---

## Khám phá dữ liệu - tập huấn luyện

In [116]:
en_overview_dup = en_train_X.duplicated(['overview'])
print("Số lượng nội dung tóm tắt bị trùng:", en_overview_dup.sum())

print("3 nội dung tóm tắt trùng nhiều nhất:")
print(en_train_X[en_overview_dup].overview.value_counts()[:3])
    

Số lượng nội dung tóm tắt bị trùng: 292
3 nội dung tóm tắt trùng nhiều nhất:
We don't have an overview translated in English. Help us expand our database by adding one.                                                                                                                                                                                                                73
Closet witch Teresa Palmer and centuries-old vampire Matthew Goode are drawn into a deadly mystery and forbidden romance when a magical book shows up in an Oxford library.                                                                                                                                 2
An emotional thrill ride through the day-to-day chaos of the city's most explosive hospital and the courageous team of doctors who hold it together. They will tackle unique new cases inspired by topical events, forging fiery relationships in the pulse-pounding pandemonium of the emergency room.     2
Name: overview, d

In [139]:
word_dict = Counter()
for words in en_train_X.overview.apply(lambda s: word_tokenize(s)):
    word_dict.update(words)
    max_leng = max(max_leng, len(words))

max_leng, word_dict.most_common(20)

(302,
 [(',', 6132),
  ('the', 5523),
  ('.', 4675),
  ('a', 3547),
  ('and', 3422),
  ('of', 3141),
  ('to', 2953),
  ('in', 1876),
  ('his', 1342),
  ('is', 1326),
  ('with', 907),
  ('her', 790),
  ('on', 785),
  ("'s", 782),
  ('an', 727),
  ('by', 721),
  ('their', 690),
  ('he', 674),
  ('for', 668),
  ('as', 666)])

In [118]:
vi_overview_dup = vi_train_X.duplicated(['overview'])
print("Số lượng nội dung tóm tắt bị trùng:", vi_overview_dup.sum())

print("3 nội dung tóm tắt trùng nhiều nhất:")
print(vi_train_X[vi_overview_dup].overview.value_counts()[:3])
    

Số lượng nội dung tóm tắt bị trùng: 120
3 nội dung tóm tắt trùng nhiều nhất:
Phim Đảo Hải Tặc - One Piece là chuyện về cậu bé Monkey D. Luffy do ăn nhầm Trái Ác Quỷ, bị biến thành người cao su và sẽ không bao giờ biết bơi. 10 năm sau sự việc đó, cậu rời quê mình và kiếm đủ 10 thành viên để thành một băng hải tặc, biệt hiệu Hải tặc Mũ Rơm.Khi đó của phiêu lưu tìm kiếm kho báu One Piece bắt đầu. Trong cuộc phiêu lưu tìm kiếm One Piece, băng Hải tặc mũ rơm phải chiến đấu với nhiều băng hải tặc xấu khác cũng muốn độc chiếm One Piece và Hải quân của Chính phủ muốn diệt trừ hải tặc. Băng Hải tặc Mũ Rơm phải trải qua biết bao nhiêu khó khăn, không lùi bước với ước mơ "Trở thành Vua Hải Tặc và chiếm được kho báu One Piece".    2
Diễn viên từng được đề cử giải Oscar William H. Macy đóng vai Frank Gallagher, một người cha độc thân của sáu người dành phần lớn thời gian rảnh của mình để uống rượu tại các quán bar. Những đứa trẻ Gallagher - dẫn đầu bởi cô con gái lớn nhất Fiona (Emmy Rossum), người

In [138]:
word_dict = Counter()
max_leng = -1
for words in vi_train_X.overview.apply(lambda s: word_tokenize(s)):
    word_dict.update(words)
    max_leng = max(max_leng, len(words))

max_leng, word_dict.most_common(20)

(240,
 [(',', 3166),
  ('.', 2720),
  ('của', 1756),
  ('một', 1617),
  ('và', 1440),
  ('là', 974),
  ('người', 965),
  ('những', 775),
  ('trong', 673),
  ('với', 646),
  ('được', 641),
  ('có', 635),
  ('đã', 597),
  ('khi', 485),
  ('phim', 459),
  ('về', 451),
  ('để', 448),
  ('thành', 444),
  ('anh', 441),
  (')', 422)])

## Tiền xử lí

In [ ]:
movie_df[movie_df.score == 0.0]

## Tiền xử lí - tập validation

TODO

---

## Mô hình hóa dữ liệu

Huấn luyện mô hình

### Hyperparameter tuning

TODO

### Đánh giá mô hình

TODO

---

## Kết luận

TODO